In [6]:
# Imports
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
import sklearn as sk
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [7]:
# Read data and construct dataframe

df = pd.read_csv('datasets/tested_molecules.csv')

df.head()

,SMILES,PKM2_inhibition,ERK2_inhibition
0,C=C(C)c1nc(N)nc(N)n1,0,0
1,C=C(Cl)COc1ccc2c(C)cc(=O)oc2c1,0,0
2,C=CCNC(=O)CCCC(=O)NCC=C,0,0
3,C=CCOn1c(=O)c(C)[n+]([O-])c2ccccc21,0,0
4,C=CCn1cc(Cl)c(=O)n(CC=C)c1=O,0,0


In [12]:
# Load the dataset
df = pd.read_csv('datasets/tested_molecules.csv')  # Replace with your file path

# Function to compute descriptors
def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        descriptors = {
            'MolWt': Descriptors.MolWt(mol),
            'LogP': Descriptors.MolLogP(mol),
            'TPSA': Descriptors.TPSA(mol),
            'NumHDonors': Descriptors.NumHDonors(mol),
            'NumHAcceptors': Descriptors.NumHAcceptors(mol),
        }
        return descriptors
    else:
        return {key: None for key in descriptor_names}

# Compute descriptors for all molecules
descriptor_names = [
    'MolWt', 'LogP', 'TPSA', 'NumHDonors', 'NumHAcceptors', 
]
df['Descriptors'] = df['SMILES'].apply(compute_descriptors)
descriptors_df = pd.json_normalize(df['Descriptors'])
df = pd.concat([df, descriptors_df], axis=1).drop(columns=['Descriptors'])
df.head()

,SMILES,PKM2_inhibition,ERK2_inhibition,MolWt,LogP,TPSA,NumHDonors,NumHAcceptors
0,C=C(C)c1nc(N)nc(N)n1,0,0,151.173,0.06910,90.71,2,5
1,C=C(Cl)COc1ccc2c(C)cc(=O)oc2c1,0,0,250.681,3.23272,39.44,0,3
2,C=CCNC(=O)CCCC(=O)NCC=C,0,0,210.277,0.76110,58.20,2,2
3,C=CCOn1c(=O)c(C)[n+]([O-])c2ccccc21,0,0,232.239,0.55792,58.17,0,4
4,C=CCn1cc(Cl)c(=O)n(CC=C)c1=O,0,0,226.663,1.03540,44.00,0,4


In [9]:
# Set the columns which we use for predicting data, so it is easy to change if ever necessary.
x_column = 'SMILES'
y_columns = ['PKM2_inhibition', 'ERK2_inhibition' ]

In [10]:
# Convert SMILES to fingerprints, so we have usable data training

def smiles_to_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
        return list(fp)
    else:
        return [0] * 1024
    
df['Fingerprint'] = df['SMILES'].apply(smiles_to_fingerprint)
x_column = 'Fingerprint'

In [11]:
# Create test and train sets

X = list(df[x_column])
y = df[y_columns].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

KeyError: "['RadiusOfGyration'] not in index"

In [ ]:
# Prediction model

# Define the pipeline
model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=0)))
])

# Train model
model.fit(X_train, y_train)

# Make prediction using trained model
predictions = model.predict(X_test)
rounded_predictions = (predictions >= 0.5).astype(int)

In [ ]:
# Calculate prediction scores

# Calculate metrics for each property
accuracy1 = accuracy_score(y_test.iloc[:, 0], rounded_predictions[:, 0])
precision1 = precision_score(y_test.iloc[:, 0], rounded_predictions[:, 0], zero_division=1)
recall1 = recall_score(y_test.iloc[:, 0], rounded_predictions[:, 0], zero_division=1)
f1_1 = f1_score(y_test.iloc[:, 0], rounded_predictions[:, 0], zero_division=1)

accuracy2 = accuracy_score(y_test.iloc[:, 1], rounded_predictions[:, 1])
precision2 = precision_score(y_test.iloc[:, 1], rounded_predictions[:, 1], zero_division=1)
recall2 = recall_score(y_test.iloc[:, 1], rounded_predictions[:, 1], zero_division=1)
f1_2 = f1_score(y_test.iloc[:, 1], rounded_predictions[:, 1], zero_division=1)

# Print the results
print("PKM2 - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1: {:.4f}".format(accuracy1, precision1, recall1, f1_1))
print("ERK2 - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1: {:.4f}".format(accuracy2, precision2, recall2, f1_2))

PKM2 - Accuracy: 0.9688, Precision: 1.0000, Recall: 0.0000, F1: 0.0000
ERK2 - Accuracy: 0.9330, Precision: 0.0000, Recall: 0.0000, F1: 0.0000
